# Channel Break Out Detection

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression

In [4]:
#df = pd.read_csv(r"D:\CARLOS\TRADING\EURUSD_Candlestick_1_D_BID_04.05.2003-21.01.2023.csv")
ini2 = date.today() - timedelta(days=728) # 2 years ago
today = date.today()
today_str = str(today)
ini2_str = str(ini2)
tickers = [
    'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [5]:
df = pd.DataFrame()
for ticker in tickers:
    company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    company['datetime'] = pd.to_datetime (company.index)
    company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Detect Pivots/Fractals

In [ ]:
ord=10
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']+1e-3
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

In [12]:
#CODIGO DE REVISION CARLOS
dfpl = df.iloc[750:950]

p = figure(width=2500, height=500,
        title="RCB",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

###### Canales bajistas, cuando el PM de 40 esta por encima del PM de 20 en direccion desendente
###### RSI>70 y sobre las olas de Bollinger

In [ ]:
def rcb01(df):
    if ((df['pivotLow']>0) & (df['SMA40']>df['SMA20'])):
        return 1
    else:
        return 0
df['rcb01_01'] = df.apply(rcb01, axis=1)

## Detect Price Channel

In [ ]:
def collect_channel(candle, backevalTrend, trendH):
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles

    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    if len(highs)>=2:
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)
    

In [ ]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    if (dfpl.loc[candle,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
        val = candle - backeval
        
    if val>75:
        backeval=candle-75
        
    if (dfpl.loc[backeval,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

## Detect Break Out

In [ ]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):

    if (candle-backeval-window)<0:
        return np.nan

    ini=candle-backeval
    fin=candle+window
    #print("candle:",candle,",backeval", backeval, ", ini:", ini, ", fin:", fin,   ", rowMin:", row.index[0],", rowMax:", row.index[-1])
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [ ]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = df.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    #trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    #trend_currlow=dfpl.loc[candleEval].trendcurrlow
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [27]:
def breakpointpos(x):
    if x['isBreakOutFinal'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutFinal']==1:
        return x['High']-3e-3
    else:
        return np.nan

##### Hallando casos de Ruptura del Canal Bajista

In [ ]:
#FUNCION DE REVISION DE VELAS

In [ ]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    if(df['rcb01_01'][i]==1): #posibles casos 
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval]       

        
        
        h=1
        l=1

        cant = cant +1
        trendH, backevalTrend = backcandle(candle)

        backevalTrend2 = candle - backevalTrend
                    
        #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
        sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                
        dfpl.loc[trendH, "trendH"] = 1
    
        
        if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
            #crear linea de tendencia high y low
            dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)
            
            ind_vela = revisionVelas(dfpl, backeval,window, candle)
            promVol = dfpl.loc[i-backeval:i]["Volume"].mean()

            #print("ind_vela:",ind_vela)
            if (ind_vela>0):
                dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                dfpl["ind_posicion"] = 0
                dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                dfpl["promVol"] = promVol
                dfpl["sl_highs"] = sl_highs
                dfpl["r_sq_h"] = r_sq_h
        
                start = dfpl.index[0]
                dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                # Solo me quedo con el primer BREAK OUT
                cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
                id=0
                if cnt>0:
                    id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]
                    dfpl['isBreakOutFinal'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])
                    dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                else:
                    dfpl['isBreakOutFinal'] = np.nan
                    dfpl['breakpointpos'] = np.nan
        
                if cnt>0:
                    #INSERT CASO
                    caso=caso+1                            
                    dfpl["caso"] = caso
                    print(i, df['companyName'][i])
                    if len(df_casos)<1:
                        df_casos = dfpl
                    else:
                        df_casos = pd.concat([df_casos, dfpl],ignore_index=False)


89 SPY
172 SPY
188 SPY
377 SPY
432 SPY
556 SPY
633 SPY
724 SPY
782 SPY
825 SPY
854 SPY
980 SPY
1306 SPY
1734 SPY
1810 SPY
2010 SPY
2119 SPY
2238 SPY
2269 SPY
2287 SPY
2332 SPY
2475 SPY
2494 SPY
2609 SPY
2708 SPY
2752 SPY
2766 SPY
2827 SPY
2954 SPY
2986 SPY
3035 SPY
3074 SPY
3155 SPY
3313 SPY
3350 SPY
3364 SPY
3445 SPY
3556 META
3613 META
3630 META
3767 META
3978 META
4042 META
4078 META
4106 META
4126 META
4267 META
4299 META
4415 META
4445 META
4620 META
5112 META
5179 META
5266 META
5307 META
5333 META
5434 META
5450 META
5483 META
5569 META
5582 META
5699 META
5760 META
5932 META
5968 META
6145 META
6164 META
6183 META
6239 META
6250 META
6312 META
6466 META
6504 META
6559 META
6751 META
6786 META
6817 META
6837 META
6865 META
6918 META
7030 AAPL
7118 AAPL
7136 AAPL
7172 AAPL
7387 AAPL
7517 AAPL
7531 AAPL
7545 AAPL
7579 AAPL
7670 AAPL
7704 AAPL
7715 AAPL
7775 AAPL
7866 AAPL
7894 AAPL
7918 AAPL
8082 AAPL
8094 AAPL
8205 AAPL
8252 AAPL
8296 AAPL
8380 AAPL
8541 AAPL
8625 AAPL
8638 AAPL


In [ ]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
